<a href="https://colab.research.google.com/github/quanvu0996/ds_test/blob/main/DS_SOLUTION_Test_T12_THUC_HANH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ĐÁNH GIÁ NĂNG LỰC KỸ SƯ KHOA HỌC DỮ LIỆU THÁNG 12 - 2021
#### BÀI THỰC HÀNH
---

Ngày test: 24/12/2021
Thời gian làm bài: 60 phút, từ 10:00 đến 11:00
(+ 5 phút thiết lập môi trường và nộp bài)

Đề thi có 12 câu, tương đương 20 điểm

Quy định:
- Không trao đổi trong suốt quá trình làm bài
- Record màn hình làm bài và share lại cho quanvh8 sau khi kết thúc thời gian test
- Được sử dụng tài liệu và tra cứu internet
- Sau khi hoàn thành, tải notebook, gửi về email: quanvh8@viettel.com.vn

Hướng dẫn làm bài:

* Chạy toàn bộ lệnh trong phần 0. CHUẨN BỊ MÔI TRƯỜNG
* Làm theo hướng dẫn tại các câu và CHỈ SỬA các đoạn code nằm giữa các thẻ:

      # YOUR CODE START
      # / END YOUR CODE

*Lưu ý: hầu hết câu hỏi không có tính tuần tự (không cần hoàn thành câu trước để làm câu sau)*

*Đề thi có một số testcase hỗ trợ DS kiểm tra kết quả, nhưng đây không phải điểm số. Điểm số được giám khảo chấm và gửi lại sau.*



---



## 0. CHUẨN BỊ MÔI TRƯỜNG 
Guide: Chạy toàn bộ các lệnh trong phần này

In [ ]:
!pip install pyspark==3.2.0

     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 198 kB 61.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=e9a1644d58471182f532955f272801e9b024b800a4badb67def955566655d24e
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import requests 
import seaborn as sns
import matplotlib.pyplot as plt

import pyspark
from pyspark.sql import functions as F

In [ ]:
print(pyspark.__version__)

3.2.0


In [ ]:
spark = pyspark.sql.SparkSession \
    .builder \
    .appName("DS_test") \
    .getOrCreate()

#### DATA
Bộ dữ liệu "Student Performance Data Set", 
Xem thêm bộ dữ liệu này tại: [UCI Student Performance](https://archive.ics.uci.edu/ml/datasets/Student+Performance)

In [ ]:
# Tải dữ liệu về
file_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip'
file_path = "/content/student.zip"

r = requests.get(file_url, stream = True) 

with open( file_path, "wb") as file: 
    for block in r.iter_content(chunk_size = 1024*10): 
        if block: 
            file.write(block)

In [ ]:
!unzip student.zip

Archive:  student.zip
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [ ]:
# TEST CASE
class TEST():
  def test_1(df_math, df_por):
    tcs = lambda df: [len(df.columns), df.count()]
    if tcs(df_math) == [33, 395]: print('df_math: PASS') 
    else : print('df_math: FAIL')
    if tcs(df_por) == [33, 649]: print('df_por: PASS') 
    else : print('df_por: FAIL')

  def test_2(sql_text):
    check = spark.sql(sql_text).toPandas().sort_values('sex')['student_num'].values
    if [check.sum(), check[0]/check[1] ] == [21, 1.1]: print('PASS')
    else: print('FAIL')

## 1. BIG DATA MANIPULATION AND ANALYSIS

Câu 1. [2p, 1đ] Đọc dữ liệu từ file

Sau khi tải về và giải nén, bộ dữ liệu gồm 2 file:
* student-mat.csv : dữ liệu điểm môn toán
* student-por.csv : dữ liệu điểm môn tiếng Bồ Đào Nha

YC: Đọc 2 file này vào các pyspark dataframe tương ứng df_math, df_por

In [ ]:
file_path_math = "/content/student-mat.csv"
file_path_por = "/content/student-por.csv"

df_math = (
    # YOUR CODE START 
    spark.read.options(header='True', inferSchema='True', delimiter=';').csv(file_path_math)
    
    # / END YOUR CODE
)

df_por = (
    # YOUR CODE START 
    spark.read.options(header='True', inferSchema='True', delimiter=';').csv(file_path_por)
    
    # / END YOUR CODE
)

In [ ]:
TEST.test_1(df_math, df_por)

df_math: PASS
df_por: PASS


### 1.1. SPARK SQL
Sử dụng SQL để thực hiện các ý sau với bảng tbl_math, và tbl_por

In [ ]:
df_math.createOrReplaceTempView('tbl_math')
df_math.createOrReplaceTempView('tbl_por')

Câu 2.[ 2p, 1 đ]
Xác định nhóm ngành nghề của mẹ (cột Mjob) nào tương ứng với điểm số cả năm (cột g3) của người con cao nhất

In [ ]:
sql_text = (
# YOUR CODE START 
'''
SELECT mjob, AVG(g3) as average_score
FROM   tbl_math
GROUP BY mjob
ORDER BY average_score DESC
LIMIT 1
'''
# / END YOUR CODE
)
spark.sql(sql_text).toPandas()

,mjob,average_score
0,health,12.147059


Câu 3. [2p, 2đ] Xác định hệ số tương quan Pearson của điểm số học kỳ 1 (cột G1) và điểm số học kỳ 2 (cột G2)

In [ ]:
sql_text = (
# YOUR CODE START 
'''
SELECT  (AVG(g1 * g2) - (AVG(g1) * AVG(g2))) / (STDDEV(g1) * STDDEV(g2)) as corr
FROM    tbl_math
'''
# / END YOUR CODE
)
spark.sql(sql_text).toPandas() 

,corr
0,0.849961


Câu 4.[ 10p, 2đ]
Sở Giáo dục dự kiến cấp học bổng cho các học sinh dựa trên kết quả học tập cả năm môn toán (bảng tbl_math), sao cho:
* Các học sinh có kết quả học tập cả năm (cột G3) cao hơn thì ưu tiên lựa chọn.
* Tỷ trọng số lượng học sinh theo từng gới tính (cột sex) được nhận học bổng là như nhau: số học sinh nam nhận học bổng trên tổng số học sinh nam bằng số học sinh nữ nhận học bổng trên tổng số nữ.
* Tổng số thời gian vắng mặt (cột absences) của các học sinh được nhận học bổng không quá 5% tổng thời gian vắng mặt của tất cả học sinh.

YC: Viết lệnh SQL truy xuất số lượng học sinh được nhận học bổng của mỗi giới tính. Bảng đầu ra có các cột: {sex:string, student_num:int}

In [ ]:
sql_text = (
# YOUR CODE START 
'''
WITH pct_table AS (
  SELECT  *, ROW_NUMBER() OVER (PARTITION BY sex ORDER BY g3 DESC)
            / COUNT(1) OVER (PARTITION BY sex) as pct
  FROM tbl_math
)
SELECT sex, COUNT(1) AS student_num
FROM  (
        SELECT  *, SUM(absences) OVER (ORDER BY pct)/ SUM(absences) OVER () as ratio
        FROM    pct_table
      )
WHERE ratio <= 0.05
GROUP BY sex
'''
# / END YOUR CODE
)
spark.sql(sql_text).toPandas()

,sex,student_num
0,F,11
1,M,10


In [ ]:
TEST.test_2(sql_text)

PASS


### 1.2. PYSPARK
Sử dụng pyspark và các dataframe df_math, df_por trả lời các câu hỏi sau

Câu 5. [2p, 1đ] Xác định số lượng học sinh vừa học môn toán, vừa học tiếng Bồ Đào Nha. Biết rằng, mỗi sinh viên được đặc trưng bởi một tổ hợp các cột: "school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"

In [ ]:
# YOUR CODE START 
student_id = ["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"]
df_por2=df_por.withColumnRenamed('g3', 'por_g3')

df_merge = df_math.join(df_por2, on = student_id)

df_merge.count()
# / END YOUR CODE

382

Câu 6. [2p, 1đ]
Xác định hệ số tương quan Pearson giữa điểm số cả năm môn toán và điểm số cả năm môn tiếng Bồ Đào Nha của các học sinh đều học 2 môn này


In [ ]:
# YOUR CODE START 

pearson_corr = df_merge.stat.corr( 'G3', 'por_G3')
pearson_corr

# / END YOUR CODE

0.4803493619410261

Câu 7. [5p. 2đ] Xác định hệ số tương quan Kendall tau của điểm số học kỳ 2 học kỳ (G1 và G2) giữa môn toán và môn tiếng Bồ Đào Nha của từng học sinh (cho các học sinh học cả 2 môn này). Biết rằng pyspark không hỗ trợ , nhưng scipy có hỗ trợ tính Kendall tau tại: scipy.stats.kendalltau.

In [ ]:
from scipy.stats import kendalltau

# YOUR CODE START 

@F.pandas_udf("float")
def kendall_corr(x: pd.Series, y: pd.Series)-> float:
    return kendalltau(x, y).correlation

df_merge.select(kendall_corr(df_merge['G3'], df_merge['por_G3'])).show()

# / END YOUR CODE

+------------------------+
|kendall_corr(G3, por_G3)|
+------------------------+
|               0.4188197|
+------------------------+



## 2. MODELING

In [ ]:
pddf_por = pd.read_csv(file_path_por, sep=';')
pddf_por.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


Câu 8. [2p, 1đ] Sử dụng pandas dataframe pddf_por, chia ngẫu nhiên thành 2 tập train, test với tỷ lệ 7:3

In [ ]:
# YOUR CODE START 

from sklearn.model_selection import train_test_split
train, test = train_test_split(pddf_por, test_size = 0.3, random_state=42)

# / END YOUR CODE

Câu 9. [5p, 3đ] Tiền xử lý dữ liệu

* Với các nhãn G1, G2, G3: giữ nguyên
* Với các features (các cột còn lại): 
  * Với cột dạng số: áp dụng StandardScaler
  * Với cột dạng yes/no: chuyển về dạng 0/1
  * Với cột dạng biến danh mục: sử dụng onehot-encoder

Fit-transform theo luồng trên cho tập train, transform cho tập test để tạo bộ feature

In [ ]:
# YOUR CODE START 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

cat_cols = [i for i in train.select_dtypes(include='object').columns]
num_cols = [i for i in train.select_dtypes(include='int64').columns if i not in ('G1', 'G2', 'G3')]

pipe = ColumnTransformer([
     ('cat', OneHotEncoder(), cat_cols),
     ('num', StandardScaler() , num_cols)
     ])

train_scaled = pipe.fit_transform(train)
test_scaled = pipe.transform(test)

# / END YOUR CODE

Câu 10. [9p, 2đ] Chọn một mô hình và huấn luyện dự đoán đồng thời 3 nhãn G1, G2, G3. Đánh giá hiệu quả model trên tập test với chỉ tiêu mean absolute error (MAE). Sao cho trung bình MAE 3 nhãn trên tập test <=1.98. 

Tính giá trị MAE từng nhãn khi đó.

In [ ]:
# YOUR CODE START 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
rf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, min_samples_leaf=4, random_state=42)
rf.fit(train_scaled, train[['G1', 'G2', 'G3']])

Y_pred = rf.predict(test_scaled)

print( mean_absolute_error( test[['G1', 'G2', 'G3']] ,Y_pred ))

print(mean_absolute_error( test[['G1', 'G2', 'G3']] ,Y_pred , multioutput= 'raw_values'))



# / END YOUR CODE

1.953846153846154
[2.02564103 1.74871795 2.08717949]


Câu 11. [9p, 2đ] Xây dựng một mạng học sâu với tối thiểu 2 hidden layers với các tham số phù hợp để dự đoán các nhãn G1, G2, G3. Sao cho trung bình MAE 3 nhãn trên tập test <=1.98. 

Tính giá trị MAE từng nhãn khi đó.

In [ ]:
# YOUR CODE START 
model = tf.keras.Sequential([
          tf.keras.layers.Dense(64, activation='relu'), 
          tf.keras.layers.Dropout(0.25), 
          tf.keras.layers.Dense(16, activation='relu'), 
          tf.keras.layers.Dense(3)
])

model.compile( optimizer='adam', loss='mae')
model.fit(train_scaled, train[['G1', 'G2', 'G3']], 
          epochs=75, validation_data = [test_scaled, test[['G1', 'G2', 'G3']] ])

print(mean_absolute_error( test[['G1', 'G2', 'G3']] ,model.predict(test_scaled) , multioutput= 'raw_values'))

# / END YOUR CODE

Epoch 1/75
15/15 [==============================] - 0s 11ms/step - loss: 11.6317 - val_loss: 11.2336
Epoch 2/75
15/15 [==============================] - 0s 3ms/step - loss: 10.8522 - val_loss: 10.2841
Epoch 3/75
15/15 [==============================] - 0s 3ms/step - loss: 9.6873 - val_loss: 8.7406
Epoch 4/75
15/15 [==============================] - 0s 3ms/step - loss: 7.8660 - val_loss: 6.6226
Epoch 5/75
15/15 [==============================] - 0s 3ms/step - loss: 6.1574 - val_loss: 5.3063
Epoch 6/75
15/15 [==============================] - 0s 5ms/step - loss: 4.9843 - val_loss: 4.0562
Epoch 7/75
15/15 [==============================] - 0s 4ms/step - loss: 3.7130 - val_loss: 2.7768
Epoch 8/75
15/15 [==============================] - 0s 4ms/step - loss: 2.6087 - val_loss: 1.9631
Epoch 9/75
15/15 [==============================] - 0s 4ms/step - loss: 2.3422 - val_loss: 1.9437
Epoch 10/75
15/15 [==============================] - 0s 5ms/step - loss: 2.3476 - val_loss: 1.9175
Epoch 11/75
15

Câu 12. [10p, 2đ] Áp dụng transfer learning, hãy sử dụng mạng vừa tạo để xây dựng mô hình dự đoán điểm môn toán.
1. Tiền xử lý dữ liệu tương tự như với bộ pddf_por
2. Đóng băng 2 layer đầu của model đã huấn luyện trên bộ Tiếng Bồ Đào Nha, bổ sung thêm ít nhất 2 layer phía sau tạo thành mô hình dự đoán 3 đầu ra môn Toán.
3. Huấn luyện mô hình dự đoán 3 điểm số môn toán (G1, G2, G3) và đánh giá bằng chỉ tiêu mean square error (MAE).Sao cho trung bình MAE 3 nhãn trên tập test <= 3.0
4. Khi đó, MAE của từng nhãn G1, G2, G3 môn toán là bao nhiêu?

In [ ]:
pddf_math = pd.read_csv(file_path_math, sep=';')
# YOUR CODE START 

train_math, test_math = train_test_split(pddf_math, test_size = 0.3, random_state=42)
pipe_math = ColumnTransformer([
     ('cat', OneHotEncoder(), cat_cols),
     ('num', StandardScaler() , num_cols)
     ])

train_math_scaled = pipe_math.fit_transform(train_math)
test_math_scaled = pipe_math.transform(test_math)

# Por model layes
layer_1 = model.layers[0]
layer_2 = model.layers[2]

layer_1.trainable=False
layer_2.trainable=False
print( len(layer_1.trainable_weights), len(layer_2.trainable_weights))

# Math model
model_math = tf.keras.Sequential([
          layer_1, layer_2,
          tf.keras.layers.Dense(8, activation='relu'),
          tf.keras.layers.Dense(3)
])

model_math.compile( optimizer='adam', loss='mae')
model_math.fit(train_math_scaled, train_math[['G1', 'G2', 'G3']], 
          epochs=75, validation_data = [test_math_scaled, test_math[['G1', 'G2', 'G3']] ])

print(mean_absolute_error(test_math[['G1', 'G2', 'G3']] ,
                          model_math.predict(test_math_scaled) , 
                          multioutput= 'raw_values'))


# / END YOUR CODE

0 0
Epoch 1/75
9/9 [==============================] - 0s 16ms/step - loss: 11.3193 - val_loss: 10.6546
Epoch 2/75
9/9 [==============================] - 0s 7ms/step - loss: 10.5420 - val_loss: 9.9492
Epoch 3/75
9/9 [==============================] - 0s 7ms/step - loss: 9.8465 - val_loss: 9.3351
Epoch 4/75
9/9 [==============================] - 0s 5ms/step - loss: 9.2374 - val_loss: 8.7984
Epoch 5/75
9/9 [==============================] - 0s 4ms/step - loss: 8.6979 - val_loss: 8.2835
Epoch 6/75
9/9 [==============================] - 0s 7ms/step - loss: 8.1987 - val_loss: 7.7949
Epoch 7/75
9/9 [==============================] - 0s 4ms/step - loss: 7.7239 - val_loss: 7.3337
Epoch 8/75
9/9 [==============================] - 0s 4ms/step - loss: 7.2727 - val_loss: 6.8904
Epoch 9/75
9/9 [==============================] - 0s 4ms/step - loss: 6.8332 - val_loss: 6.4547
Epoch 10/75
9/9 [==============================] - 0s 4ms/step - loss: 6.4034 - val_loss: 6.0227
Epoch 11/75
9/9 [==============